In [1]:
import numpy as np
import pandas as pd

In [21]:
train = pd.read_csv("E:/Competition/Datafest/train.csv")

In [45]:
test = pd.read_csv("E:/Competition/Datafest/test.csv")

In [24]:
train.head()

,ID,Volume,Three_Day_Moving_Average,Five_Day_Moving_Average,Ten_Day_Moving_Average,Twenty_Day_Moving_Average,True_Range,Average_True_Range,Positive_Directional_Movement,Negative_Directional_Movement,Outcome
0,1_1,-0.33308,-0.99681,-0.98736,-0.94728,-0.91977,-0.21129,-0.38778,-0.39903,-0.14321,1
1,3_1,-0.67096,-2.08895,-2.08251,-2.02183,-1.93779,-0.46476,0.97308,-0.31078,0.80286,0
2,4_1,-0.22040,1.07839,1.09143,1.18737,NaN,-0.51638,-0.52368,-0.24140,-0.31910,0
3,6_1,-0.23118,-0.55106,-0.53976,-0.47569,-0.42449,-0.60816,-0.69311,-0.40673,-0.45965,0
4,10_1,0.15076,-1.40937,-1.39805,-1.36756,-1.31365,-0.33206,-0.43690,-0.33347,0.29217,0


In [23]:
train.drop(train.columns[[ 1, 2]], axis=1, inplace= True)

In [34]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101946 entries, 0 to 101945
Data columns (total 12 columns):
ID                               101946 non-null object
timestamp                        101946 non-null int64
Stock_ID                         101946 non-null int64
Volume                           101946 non-null float64
Three_Day_Moving_Average         101892 non-null float64
Five_Day_Moving_Average          101819 non-null float64
Ten_Day_Moving_Average           101666 non-null float64
Twenty_Day_Moving_Average        101359 non-null float64
True_Range                       101946 non-null float64
Average_True_Range               101545 non-null float64
Positive_Directional_Movement    101846 non-null float64
Negative_Directional_Movement    101885 non-null float64
dtypes: float64(9), int64(2), object(1)
memory usage: 9.3+ MB


In [28]:
x_train.head()

,Volume,Three_Day_Moving_Average,Five_Day_Moving_Average,Ten_Day_Moving_Average,Twenty_Day_Moving_Average,True_Range,Average_True_Range,Positive_Directional_Movement,Negative_Directional_Movement
0,-0.33308,-0.99681,-0.98736,-0.94728,-0.91977,-0.21129,-0.38778,-0.39903,-0.14321
1,-0.67096,-2.08895,-2.08251,-2.02183,-1.93779,-0.46476,0.97308,-0.31078,0.80286
2,-0.22040,1.07839,1.09143,1.18737,NaN,-0.51638,-0.52368,-0.24140,-0.31910
3,-0.23118,-0.55106,-0.53976,-0.47569,-0.42449,-0.60816,-0.69311,-0.40673,-0.45965
4,0.15076,-1.40937,-1.39805,-1.36756,-1.31365,-0.33206,-0.43690,-0.33347,0.29217


In [25]:
x_train = train
y_train = train['Outcome'].values

In [27]:
x_train.drop('Outcome', axis=1, inplace = True)
x_train.drop('ID', axis=1, inplace = True)

In [29]:
x_train.head()

,Volume,Three_Day_Moving_Average,Five_Day_Moving_Average,Ten_Day_Moving_Average,Twenty_Day_Moving_Average,True_Range,Average_True_Range,Positive_Directional_Movement,Negative_Directional_Movement
0,-0.33308,-0.99681,-0.98736,-0.94728,-0.91977,-0.21129,-0.38778,-0.39903,-0.14321
1,-0.67096,-2.08895,-2.08251,-2.02183,-1.93779,-0.46476,0.97308,-0.31078,0.80286
2,-0.22040,1.07839,1.09143,1.18737,NaN,-0.51638,-0.52368,-0.24140,-0.31910
3,-0.23118,-0.55106,-0.53976,-0.47569,-0.42449,-0.60816,-0.69311,-0.40673,-0.45965
4,0.15076,-1.40937,-1.39805,-1.36756,-1.31365,-0.33206,-0.43690,-0.33347,0.29217


In [30]:
# Finally, we split some of the data off for validation

from sklearn.cross_validation import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)

In [31]:
import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.11 #0.02
params['max_depth'] = 5
#params['subsample']=0.7
#params['min_child_weight']=1
#params['colsample_bytree']=0.7
params['silent']=1
#params['seed']=1632

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)

[0]	train-logloss:0.690945	valid-logloss:0.691077
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.6822	valid-logloss:0.683193
[20]	train-logloss:0.680369	valid-logloss:0.681874
[30]	train-logloss:0.679525	valid-logloss:0.681456
[40]	train-logloss:0.679018	valid-logloss:0.681306
[50]	train-logloss:0.678585	valid-logloss:0.681209
[60]	train-logloss:0.678167	valid-logloss:0.681089
[70]	train-logloss:0.677838	valid-logloss:0.681
[80]	train-logloss:0.677593	valid-logloss:0.680972
[90]	train-logloss:0.677291	valid-logloss:0.680913
[100]	train-logloss:0.676936	valid-logloss:0.680847
[110]	train-logloss:0.676682	valid-logloss:0.680831
[120]	train-logloss:0.676475	valid-logloss:0.680845
[130]	train-logloss:0.676194	valid-logloss:0.680823
[140]	train-logloss:0.675983	valid-logloss:0.680818
[150]	train-logloss:0.675803	valid-logloss:0.680825
[160]	train-logloss:0.675566	vali

In [46]:
test.head()
df_test = pd.DataFrame()
df_test['ID'] =  test['ID']

In [48]:
test.drop(test.columns[[ 0,1,2]], axis=1, inplace= True)

In [49]:
test.head()

,Volume,Three_Day_Moving_Average,Five_Day_Moving_Average,Ten_Day_Moving_Average,Twenty_Day_Moving_Average,True_Range,Average_True_Range,Positive_Directional_Movement,Negative_Directional_Movement
0,0.43959,-1.09885,-1.08146,-1.02367,-0.95527,0.11595,0.17058,-0.39903,-0.33365
1,-0.38494,-0.64131,-0.64340,-0.63477,-0.61062,-0.47816,-0.48696,-0.28545,-0.30166
2,1.38417,-0.53956,-0.46261,-0.22662,-0.15185,0.27538,0.86787,-0.31078,-0.43770
3,-0.22040,-1.64317,-1.66702,-1.66951,-1.58663,-0.51638,0.04259,-0.24140,-0.31910
4,-0.21500,0.14539,0.12088,0.15728,0.13933,0.10491,-0.63238,-0.40673,-0.45965


In [50]:
d_test = xgb.DMatrix(test)
p_test = bst.predict(d_test)

In [51]:
sub = pd.DataFrame()
sub['ID'] = df_test['ID']
sub['Outcome'] = p_test
sub.to_csv('E:/Competition/Datafest/simple_xgb.csv', index=False)